<a href="https://colab.research.google.com/github/JALVARADORUIZ/TFM_UB/blob/main/version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd
import cv2
import glob
import os
import matplotlib.pyplot as plt
import string
from mlxtend.plotting import plot_decision_regions
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.svm import SVC

from tqdm import tqdm
tqdm.pandas()


dim = 100

In [14]:
data = '/content/drive/MyDrive/TFM_Máster BDDS/datawine/df_img_processed2.csv'
df = pd.read_csv(data)

In [15]:
df.columns[0:15]

Index(['winery', 'product', 'region', 'country_code', 'rating', 'variety',
       'price_usd', 'image', 'continente', 'country_code_clean', 'price_cat',
       'rating_cat', 'quality', 'color', 'image_clean'],
      dtype='object')

In [16]:
df['image_clean']

0       MBo5T9ZQQQGbzsRmVORp-w_pl_480x640.png
1       dDyTwzIfQC68MCUgjmYWpw_pl_480x640.png
2       t44NPpbUT6uy0vRt4VD2RQ_pl_480x640.png
3       v1RahJe2QQyp4oAFOv90bA_pl_480x640.png
4       VlMihlZUQE6US7fr09w5ZA_pl_480x640.png
                        ...                  
9873    mpvstrRZSD-yVA3__GEWng_pl_480x640.png
9874    jYq_vdJgRCGsVqvs5vX3fg_pl_480x640.png
9875    4PY0biY6St2tFoytqwopJg_pl_480x640.png
9876    9YJV02cwSriLjK1al5G6DA_pl_480x640.png
9877    jUL_jvtVTOui84oN75hFXg_pl_480x640.png
Name: image_clean, Length: 9878, dtype: object

In [23]:
def getFeatures(files):
    images = []
    path = "/content/drive/MyDrive/TFM_Máster BDDS/datawine/vintage-images/"
    for f in enumerate(files):
        #print(f[1])
        image_path = path + f[1]
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (dim, dim))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        images.append(image)
    return images

X = getFeatures(df['image_clean'])

In [6]:
X = np.array(X)
X.shape

NameError: ignored

In [28]:
df_variety = df['variety'].value_counts().rename_axis('variety').to_frame('counts')
df_variety_selected = df_variety[:15]
list_variety = list(df_variety_selected.index)

def transform_variety(variety, list_variety):
  if variety in list_variety:
    return variety
  else:
    return "other"

df['variety'] = df.progress_apply(lambda x:transform_variety(x.variety,list_variety), axis=1)

le = LabelEncoder()
variedad = le.fit_transform(df['variety'])
y = np.array(variedad)

100%|██████████| 9878/9878 [00:02<00:00, 4481.16it/s]


In [29]:
y.shape

(9878,)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [4]:
#Scale Data Images
scaler = StandardScaler()
X_train = scaler.fit_transform([i.flatten() for i in X_train])
X_test = scaler.fit_transform([i.flatten() for i in X_test])
X_scaled = scaler.fit_transform([i.flatten() for i in X])

NameError: ignored

Busqueda de n_components PCA

In [3]:
#Instanciamos objeto PCA y aplicamos
pca=PCA(n_components=100) 
pca.fit(X_scaled) # obtener los componentes principales
X_pca=pca.transform(X_scaled) # convertimos nuestros datos con las nuevas dimensiones de PCA
 
#print("shape of X_pca", X_pca.shape)
expl = pca.explained_variance_ratio_
#print(expl)
print('suma:',sum(expl[0:100]))

 
#graficamos el acumulado de varianza explicada en las nuevas dimensiones
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

NameError: ignored